In [1]:
import scvelo as scv
import pandas as pd
import numpy as np
import scanpy as sc
import glob

In [2]:
sample = 'ncc'
results_file = '/home/linxy29/holab/iPSC/veloAE/' + sample + '_seurate.h5ad'  # the file that will store the analysis results

In [3]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 pynndescent==0.5.7


In [4]:
# get meta-data information
path = "/home/linxy29/holab/iPSC/veloAE/"
cellID_obs = pd.read_csv(path + sample + '_cellID_obs.csv')
print(cellID_obs.head())
print(len(cellID_obs))
umap_cord = pd.read_csv(path + sample + '_cell_embeddings.csv')
print(umap_cord)
cell_clusters = pd.read_csv(path + sample + '_clusters.csv')

                    x
0  AAACCCATCAGCAATC-1
1  AAACGAACACGGTGCT-1
2  AAACGAACATTGTCGA-1
3  AAACGCTCAAGAGGTC-1
4  AAACGCTGTGCCCGTA-1
2533
              Unnamed: 0    UMAP_1    UMAP_2
0     AAACCCATCAGCAATC-1 -2.503784  3.151836
1     AAACGAACACGGTGCT-1 -1.282910 -1.604800
2     AAACGAACATTGTCGA-1 -1.163274 -4.289365
3     AAACGCTCAAGAGGTC-1 -1.973931 -3.207492
4     AAACGCTGTGCCCGTA-1  1.331261  5.161453
...                  ...       ...       ...
2528  TTTGGTTGTTTGCCGG-1 -0.638352  3.165435
2529  TTTGGTTTCTCCTACG-1  1.173808  5.169805
2530  TTTGTTGAGCATGATA-1 -3.560466  0.789341
2531  TTTGTTGCACACGTGC-1  1.230687  2.120469
2532  TTTGTTGGTTTCCAAG-1  4.166612 -0.787386

[2533 rows x 3 columns]


In [5]:
## add mes data
ldata = scv.read('/home/linxy29/holab/iPSC/' + sample + '_cellranger/velocyto/' + sample + '_cellranger.loom', cache=True)
obs_name_rep = ldata.obs_names.str.replace('ncc_cellranger:|x', '') + '-1'
ldata.obs_names = obs_name_rep
ldata.var_names_make_unique()
print(ldata)
print(ldata.obs_names)

... writing an h5ad cache file to speedup reading next time
AnnData object with n_obs × n_vars = 2631 × 36601
    obs: 'Clusters', '_X', '_Y'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'
Index(['AAAGTCCTCTTAAGGC-1', 'AACAAAGCAGCTCTGG-1', 'AAAGGTAGTCATCCCT-1',
       'AACCCAATCCGTGACG-1', 'AACCATGAGATGGTCG-1', 'AAGATAGGTCATTGCA-1',
       'AAGGAATCATAGATCC-1', 'AACCATGTCGCACGAC-1', 'AAGCATCTCGTAATGC-1',
       'AACAAAGAGTGAGCCA-1',
       ...
       'TTTGACTAGGCACCAA-1', 'TTTCGATTCGAGCCTG-1', 'TTTGACTTCAAGTGTC-1',
       'TTTGTTGAGCATGATA-1', 'TTTCGATAGTGTCATC-1', 'TTTATGCTCTCCATAT-1',
       'TTTATGCAGCTACTGT-1', 'TTTGACTGTGCCCAGT-1', 'TTTCGATTCTTTCCAA-1',
       'TTTCGATTCCGTGACG-1'],
      dtype='object', name='CellID', length=2631)


## barcodes

In [6]:
## change obs_names
print("Whether the barcodes are unique:")
print(np.unique(ldata.obs_names).size == len(ldata.obs_names))
print("The number of barcode:")
print(len(ldata.obs_names))

Whether the barcodes are unique:
True
The number of barcode:
2631


In [7]:
filtered_barcode = np.intersect1d(cellID_obs['x'],ldata.obs_names)
len(filtered_barcode)

2533

In [8]:
filtered_ldata = ldata[filtered_barcode].copy()
print(filtered_ldata)

AnnData object with n_obs × n_vars = 2533 × 36601
    obs: 'Clusters', '_X', '_Y'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'


Add UMAP and annotation

In [9]:
ldata_index = pd.DataFrame(filtered_ldata.obs.index)
ldata_index.columns = ['CellID']
print(ldata_index)

                  CellID
0     AAACCCATCAGCAATC-1
1     AAACGAACACGGTGCT-1
2     AAACGAACATTGTCGA-1
3     AAACGCTCAAGAGGTC-1
4     AAACGCTGTGCCCGTA-1
...                  ...
2528  TTTGGTTGTTTGCCGG-1
2529  TTTGGTTTCTCCTACG-1
2530  TTTGTTGAGCATGATA-1
2531  TTTGTTGCACACGTGC-1
2532  TTTGTTGGTTTCCAAG-1

[2533 rows x 1 columns]


In [10]:
umap_cord = umap_cord.rename(columns = {'Unnamed: 0':'CellID'})
#umap_cord = umap_cord.rename(columns = {'Cell ID':'CellID'})
umap_ordered = ldata_index.merge(umap_cord, on = "CellID")
print(umap_cord)

                  CellID    UMAP_1    UMAP_2
0     AAACCCATCAGCAATC-1 -2.503784  3.151836
1     AAACGAACACGGTGCT-1 -1.282910 -1.604800
2     AAACGAACATTGTCGA-1 -1.163274 -4.289365
3     AAACGCTCAAGAGGTC-1 -1.973931 -3.207492
4     AAACGCTGTGCCCGTA-1  1.331261  5.161453
...                  ...       ...       ...
2528  TTTGGTTGTTTGCCGG-1 -0.638352  3.165435
2529  TTTGGTTTCTCCTACG-1  1.173808  5.169805
2530  TTTGTTGAGCATGATA-1 -3.560466  0.789341
2531  TTTGTTGCACACGTGC-1  1.230687  2.120469
2532  TTTGTTGGTTTCCAAG-1  4.166612 -0.787386

[2533 rows x 3 columns]


In [11]:
umap_ordered = umap_ordered.iloc[:,1:]
filtered_ldata.obsm['X_umap'] = umap_ordered.values

In [12]:
cell_clusters = cell_clusters.rename(columns = {'Unnamed: 0':'CellID'})
cell_clusters = ldata_index.merge(cell_clusters, on = "CellID")
print(cell_clusters)

                  CellID                          x
0     AAACCCATCAGCAATC-1                 Sclerotome
1     AAACGAACACGGTGCT-1                 Sclerotome
2     AAACGAACATTGTCGA-1      axial skeleton system
3     AAACGCTCAAGAGGTC-1                 Sclerotome
4     AAACGCTGTGCCCGTA-1  lateral/paraxial mesoderm
...                  ...                        ...
2528  TTTGGTTGTTTGCCGG-1  lateral/paraxial mesoderm
2529  TTTGGTTTCTCCTACG-1  lateral/paraxial mesoderm
2530  TTTGTTGAGCATGATA-1                 Sclerotome
2531  TTTGTTGCACACGTGC-1           Somitic mesoderm
2532  TTTGTTGGTTTCCAAG-1           Somitic mesoderm

[2533 rows x 2 columns]


In [13]:
cell_clusters = cell_clusters.iloc[:,1:]
filtered_ldata.obs['seurat_clusters'] = cell_clusters.values
print(filtered_ldata)

AnnData object with n_obs × n_vars = 2533 × 36601
    obs: 'Clusters', '_X', '_Y', 'seurat_clusters'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    obsm: 'X_umap'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'


In [25]:
filtered_ldata.write(results_file)